# CVBS Video Playback

This notebook plays back captured CVBS data through RF output 1 using Deep Memory Generation (DMG).

## Prerequisites
- Captured data file: `/tmp/cvbs_capture.npy`
- Red Pitaya OS 2.07-48 or later (for DMG support)

## Hardware Setup
- Connect RF OUT1 to your video display/capture device
- Output is ±1V range (matches CVBS levels)

## 1. Initialize Libraries and FPGA

In [ ]:
import time
import os
import gc
import numpy as np
from rp_overlay import overlay
import rp

# Initialize FPGA overlay and RP library
fpga = overlay()
rp.rp_Init()
print("Red Pitaya initialized")

# Check available memory
try:
    with open('/proc/meminfo', 'r') as f:
        for line in f:
            if 'MemAvailable' in line or 'MemFree' in line:
                print(line.strip())
except:
    pass

## 2. Load Captured Data (Memory Efficient)

In [ ]:
# Load the captured CVBS data
capture_file = "/tmp/cvbs_capture.npy"
metadata_file = "/tmp/cvbs_capture_metadata.npy"

if not os.path.exists(capture_file):
    raise FileNotFoundError(f"Capture file not found: {capture_file}\nRun cvbs_capture.ipynb first!")

# Use memory mapping to avoid loading entire file into RAM initially
raw_data = np.load(capture_file, mmap_mode='r')
num_samples = len(raw_data)
print(f"Capture file: {capture_file}")
print(f"Samples: {num_samples:,}")
print(f"Data type: {raw_data.dtype}")
print(f"Min: {raw_data.min()}, Max: {raw_data.max()}")

# Load metadata if available
SAMPLE_RATE = 15.625e6  # Default
if os.path.exists(metadata_file):
    try:
        metadata = np.load(metadata_file, allow_pickle=True).item()
        SAMPLE_RATE = metadata.get('sample_rate', SAMPLE_RATE)
        print(f"Sample rate from metadata: {SAMPLE_RATE/1e6:.3f} MS/s")
    except:
        print(f"Could not load metadata, using default sample rate")

duration = num_samples / SAMPLE_RATE
print(f"Duration: {duration:.3f} seconds")

## 3. Check DMG Availability and Memory

In [ ]:
# List available generation functions
gen_axi_funcs = [f for f in dir(rp) if 'GenAxi' in f]
print("Available GenAxi functions:")
for f in sorted(gen_axi_funcs):
    print(f"  {f}")

dmg_available = len(gen_axi_funcs) > 0
print(f"\nDMG Available: {dmg_available}")

if dmg_available and hasattr(rp, 'rp_GenAxiGetMemoryRegion'):
    try:
        memRegion = rp.rp_GenAxiGetMemoryRegion()
        print(f"Memory region result: {memRegion}")
    except Exception as e:
        print(f"Error getting memory region: {e}")
        dmg_available = False

## 4. Configure Playback (Standard AWG Fallback)

If DMG is not available, we'll use the standard AWG with a small buffer.
This will only play back a short segment (~1ms) in a loop.

In [ ]:
channel = rp.RP_CH_1
DECIMATION = 8  # 125 MS/s / 8 = 15.625 MS/s

# Reset generator
rp.rp_GenReset()
gc.collect()  # Free any unused memory

# For now, use standard AWG with limited buffer
# This plays a short segment in a loop
N = 16384  # AWG buffer size limit

print(f"Using standard AWG (buffer size: {N} samples)")
print(f"This represents {N/SAMPLE_RATE*1e6:.1f} µs of video (~{N/SAMPLE_RATE/63.5e-6:.1f} lines)")

# Load just the first N samples and convert to float in-place
print(f"\nLoading {N} samples from capture...")
segment = raw_data[:N].astype(np.float32) / 8192.0
segment = np.clip(segment, -1.0, 1.0)

print(f"Segment voltage range: {segment.min():.3f}V to {segment.max():.3f}V")

In [ ]:
# Create AWG buffer and copy data
print("Creating AWG buffer...")
buff = rp.arbBuffer(N)

print("Copying data to buffer...")
for i in range(N):
    buff[i] = float(segment[i])

# Free the numpy array
del segment
gc.collect()

print("Buffer ready")

In [ ]:
# Configure arbitrary waveform generation
print("Configuring AWG...")

rp.rp_GenWaveform(channel, rp.RP_WAVEFORM_ARBITRARY)
rp.rp_GenArbWaveform(channel, buff.cast(), N)

# Set frequency so each sample is output at the correct rate
# The AWG cycles through the buffer at (freq * N) samples per second
# We want 15.625 MS/s, so freq = 15.625e6 / N
freq = SAMPLE_RATE / N
rp.rp_GenFreqDirect(channel, freq)
print(f"Frequency: {freq:.2f} Hz (buffer repeats at this rate)")

# Set amplitude (1.0 = full scale ±1V)
rp.rp_GenAmp(channel, 1.0)

# Set DC offset to 0
rp.rp_GenOffset(channel, 0.0)

print("AWG configured")

## 5. Start Playback

In [ ]:
# Enable output
print("Starting playback...")
rp.rp_GenOutEnable(channel)

print(f"\n=== Playback active on RF OUT 1 ===")
print(f"Playing {N} samples ({N/SAMPLE_RATE*1e6:.1f} µs) in continuous loop")
print(f"Effective sample rate: {SAMPLE_RATE/1e6:.3f} MS/s")
print("\nRun next cell to stop playback")

## 6. Stop Playback

In [ ]:
# Stop playback
print("Stopping playback...")
rp.rp_GenOutDisable(channel)
print("Playback stopped")

## 7. Cleanup

In [ ]:
# Release resources
rp.rp_GenReset()
rp.rp_Release()
gc.collect()
print("Resources released")